In [83]:
import json
import os
import boto3

In [78]:
out_dir = os.getcwd()

In [17]:
metadata_file = os.path.join(out_dir, 'metadata\\metadata_landcover_esa.json')

# Load input data

In [79]:
# get list of urbanshift cities
boundary_georef = pd.read_csv('https://cities-urbanshift.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary_georef.csv') 
boundary_georef = boundary_georef.dropna(subset=['units_boundary_name']).reset_index(drop=True)
print(boundary_georef.to_string())

             geo_name   level aoi_boundary_name units_boundary_name
0         ARG-Mendoza  region         ADM3union                ADM3
1   ARG-Mar_del_Plata    city              ADM3                ADM4
2         ARG-Ushuaia    city              ADM4                ADM5
3           ARG-Salta  region         ADM2union                ADM3
4    ARG-Buenos_Aires  region         ADM2union                ADM2
5        BRA-Teresina    city         ADM4union                ADM4
6        BRA-Teresina  region         ADM2union                ADM2
7   BRA-Florianopolis    city         ADM4union                ADM4
8   BRA-Florianopolis  region         ADM2union                ADM2
9           BRA-Belem    city         ADM4union                ADM4
10          BRA-Belem  region         ADM2union                ADM2
11       CRI-San_Jose  region         ADM2union                ADM2
12         RWA-Kigali    city         ADM4union                ADM4
13       SLE-Freetown    city     ADM4cityunion 

# Create metadata file

In [80]:
dataset_metadata_cities = []

In [81]:
for i in range(0, len(boundary_georef)):
    boundary_id = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    geo_name = boundary_georef.loc[i, 'geo_name']
    #print(geo_name)
    print(boundary_id)
    
    # fill metadata fields
    dataset_title = "ESA World cover land cover extract for " + geo_name
    dataset_city_name = geo_name
    dataset_description = "The European Space Agency (ESA) WorldCover 10 m 2020 product provides a global land cover map for 2020 at 10 m resolution based on Sentinel-1 and Sentinel-2 data. The WorldCover product comes with 11 land cover classes, aligned with UN-FAO’s Land Cover Classification System, and has been generated in the framework of the ESA WorldCover project. The World Cover product comes with 11 land cover classes: Tree cover, Shrubland, Grassland, Cropland, Built-up, Bare / sparse vegetation, Snow and ice, Open water, Herbaceous wetland, Mangroves, Moss and lichen."
    dataset_snippet = "Land cover data for "+ geo_name
    dataset_tags = [geo_name,
                    "Biodiversity",
                    "Land cover",
                    "Geography:America",
                    "Geography:America:Costa_Rica",
                    "Geography:America:Costa_Rica:San_Jose",
                    "Time:2020"]
    dataset_url = "https://cities-urbanshift.s3.eu-west-3.amazonaws.com/data/land_use/esa_world_cover/v_0/"+boundary_id+"-ESA-world_cover-2000.tif"
    
    # create a dictionary
    dataset_metadata_city = {
        'title': dataset_title,
        'city_name': dataset_city_name,
        'description': dataset_description,
        'snippet': dataset_snippet,
        'tags': dataset_tags,
        'url': dataset_url
    }
    
    # append cities
    dataset_metadata_cities.append(dataset_metadata_city)
    

ARG-Mendoza-ADM3union
ARG-Mar_del_Plata-ADM3
ARG-Ushuaia-ADM4
ARG-Salta-ADM2union
ARG-Buenos_Aires-ADM2union
BRA-Teresina-ADM4union
BRA-Teresina-ADM2union
BRA-Florianopolis-ADM4union
BRA-Florianopolis-ADM2union
BRA-Belem-ADM4union
BRA-Belem-ADM2union
CRI-San_Jose-ADM2union
RWA-Kigali-ADM4union
SLE-Freetown-ADM4cityunion
SLE-Freetown-ADM4regionunion
MAR-Marrakech-ADM2
IND-Chennai-ADM4union
IND-Chennai-ADM6union
IND-Pune-ADM4union
IND-Surat-ADM4union
CHN-Chengdu-ADM3union
CHN-Chongqing-ADM1
CHN-Ningbo-ADM3union
IDN-Jakarta-ADM4union
IDN-Bitung-ADM2
IDN-Semarang-ADM1
IDN-Balikpapan-ADM4union
IDN-Palembang-ADM2


In [82]:
with open(metadata_file, 'w') as f:
    json.dump(dataset_metadata_cities, f)

In [85]:
metadata_file

'C:\\Users\\Saif.Shabou\\OneDrive - World Resources Institute\\Documents\\UrbanShift\\github\\cities-urbanshift\\geospatial-layers\\notebooks\\metadata\\metadata_landcover_esa.json'

# Upload to aws s3

In [84]:
# connect to s3
aws_credentials = pd.read_csv('C:\\Users\\Saif.Shabou\\OneDrive - World Resources Institute\\Documents\\aws\\credentials.csv')
aws_key = aws_credentials.iloc[0]['Access key ID']
aws_secret = aws_credentials.iloc[0]['Secret access key']

s3 = boto3.resource(
    service_name='s3',
    aws_access_key_id=aws_key,
    aws_secret_access_key=aws_secret
)

# specify bucket name
bucket_name = 'cities-urbanshift' 

In [86]:
# upload in s3
s3.meta.client.upload_file(metadata_file, 
                           bucket_name, 
                           'data/land_use/esa_world_cover/v_0/metadata.json',
                           ExtraArgs={'ACL':'public-read'})